In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import backend as K
import tensorflow as tf
import h5py
import os

In [2]:
tf.compat.v1.disable_eager_execution()
os.chdir('/content/drive/My Drive/github/NNanobody/')

In [3]:
IsAnneal=True
avoidnc=True
buff_range=10
constraint='oh'
saveh5=False
amino=np.asarray(['I', 'L', 'V', 'F', 'M', 'C', 'A', 'G', 'P', 'T', 'S', 'Y', 'W', 'Q', 'N', 'H', 'E', 'D', 'K', 'R', 'X'])

In [4]:
def mat2oh(datain,trim1,):
  orig_seq=[]
  trims=np.bool_(trim1)
  onehot=datain.reshape(datain.shape[0],datain.shape[1],datain.shape[3])
  colrank=np.argsort(onehot,axis=1)
  colind=np.argmax(onehot,axis=1)
  for i in range(colind.shape[0]):
    temp=colind[i][trims[i]]
    temp[onehot[i][0][trims[i]]==0.05]=20
    sequence=''.join(amino[temp])
    orig_seq.append(sequence)
  orig_seq=np.asarray(orig_seq)
  return orig_seq

In [5]:
steps = [0.005, 0.0005, 0.001, 5e-05, 0.0001]
iterations = [10, 20, 30, 40, 50, 60, 70]

In [7]:
input_all.shape

NameError: ignored

In [6]:
fi = h5py.File('./data/seeds/seeds_embedded.h5.batch1', 'r')
dataset = np.asarray(fi['data'])
dataset.shape

(6728, 20, 1, 20)

In [ ]:
for step in steps:
  for iteration in iterations:
    dataset = 'Hold out Regression'
    model_name = 'seq_32x1_16_filt3'
    outname = f'{dataset}_{model_name}_{step}_{iteration}.tsv'

    model = keras.models.load_model(f'./weights/regression/{dataset}/{model_name}')
    layer_input = model.input
    layer_size = model.input_shape
    output_layer = model.output

    input_all=np.asarray([]).reshape((0,layer_size[1],layer_size[2],layer_size[3]))
    fi = h5py.File('./data/seeds/seeds_embedded.h5.batch1', 'r')
    dataset = np.asarray(fi['data'])
    label = np.array(fi['label'])
    input_all = np.append(input_all, dataset, axis=0)

    best_map = np.empty((0,layer_size[1],layer_size[2],layer_size[3]))
    # record_all = 
    best_map=np.asarray([]).reshape((0,layer_size[1],layer_size[2],layer_size[3]))
    record_all=np.asarray([]).reshape((0,layer_size[1],layer_size[2],layer_size[3]))
    record_seed=np.asarray([])#seed for each proposed sequence
    oh_map=np.asarray([]).reshape((0,layer_size[1],layer_size[2],layer_size[3]))
    oh_act=np.asarray([])
    oh_seq=np.asarray([])
    final_act=np.asarray([])
    record_act=np.asarray([])
    record_sact=np.asarray([])
    best_act=np.asarray([])
    init_act=np.asarray([])
    init_loss=np.asarray([])
    best_it = np.asarray([])
    convg_it = np.asarray([])

    for batch in range(0,input_all.shape[0],2000):
      input_data = input_all[batch: batch + 2000] #taking 2000 chunks
      datain = np.copy(input_data)
      tryi = input_data.shape[0]

      best_input = np.zeros(input_data.shape)
      record_all_seq = np.asarray([]).reshape(0,datain.shape[1],datain.shape[2],datain.shape[3])
      record_all_act = np.asarray([])
      record_all_orig = np.asarray([])
      record_seed_act = np.asarray([])
      convg_input = np.zeros(input_data.shape)
      best_activation = np.asarray([-1.0]*tryi) # np.ones(tryi) * -1.0
      best_loss = np.asarray([-1000000000.0]*tryi) # np.ones(tryi) * -100000.0
      convg_activation = np.asarray([-10000.0]*tryi) # np.ones(tryi) * -10000.0
      best_iter = np.asarray([-1]*tryi)# np.ones(tryi) * -1.0
      convg_iter = np.asarray([-1]*tryi)# np.ones(tryi) * -1.0
      loss_track = []
      activation_track = []
      count = 0
      lr = step
      mask = np.array([False for i in range(tryi)]) # np.zeros(tryi).astype(bool)
      trim1 = np.sum(datain.reshape(datain.shape[0],datain.shape[1],datain.shape[3]),axis=1) != 0 # checks to see where the sum is != 0, false = padding, true = AA, amino_acid_mask
      trim2 = np.tile(~trim1,(1,20)).reshape(input_data.shape[0],input_data.shape[1],input_data.shape[2],input_data.shape[3]) # mask for array, padding_mask
      record_trim = np.asarray([]).reshape(0,trim1.shape[1]) 
      orig_seq = np.asarray(mat2oh(datain,trim1))
      # print(trim1.shape)
      # print(trim2.shape)

      activation = K.sum(output_layer[:,0])
      activation_all = output_layer[:,0]
      loss = activation
      loss_all = activation_all
      grads = K.gradients(loss, layer_input)[0]
      iterate = K.function([layer_input], grads)
      iterate2= K.function([layer_input],[activation_all,loss_all])
      activation_init,loss_init= iterate2([input_data])
      holdcnt = np.asarray([0]*tryi)

      print('initial activation', np.mean(activation_init))
      print('initial loss', np.mean(loss_init))  

      while True:
        for grow in range(iteration):
            grads_value = iterate([input_data,0])
            grads_value[mask,:,:,:] = 0 # don't add to values in buffer > 10
            if count > 100:
              lr = max(step*(count-100)**(-0.2),1e-6)
            input_data += grads_value*lr
            input_data[trim2] = 0.0 # set padding back to 0

            activation_all,loss_all=iterate2([input_data,0])

            onehot = input_data.reshape(input_data.shape[0],input_data.shape[1],input_data.shape[3])
            colrank = np.argsort(onehot,axis=1) # rank amino acids post gradient ascent

            # print(colrank)
            colind=np.argmax(onehot,axis=1) # index of highest gradient AA
            # print(colind)
            oh=np.zeros(onehot.shape)

        for x in range(onehot.shape[0]):
          for y in range(onehot.shape[2]):
            if avoidnc:
              if amino[colind[x,y]]=='N' or amino[colind[x,y]]=='C':
                colind[x,y]=colrank[x,-2,y] # 2nd to last row (2nd best gradient)
            oh[x,colind[x,y],y]=1.0 # assign 1 to maximum gradient AA

        oh=oh.reshape(oh.shape[0],layer_size[1],layer_size[2],layer_size[3])
        oh[trim2]=0.0
        oh_activation,oh_loss=iterate2([oh,0])
        # print('One hot Activation',np.mean(oh_activation))

        activation_track.append(oh_activation)
        temp_act=np.copy(oh_activation)
        temp_act[mask] = -10000.0 # gives huge penalty if in mask
        improve=(temp_act>best_activation) # takes where it improves

        if sum(improve)>0:
          best_activation[improve] = oh_activation[improve] # overwrite improved with value from oh_activation
          best_input[improve,:,:,:] = oh[improve,:,:,:] # copies best input from oh to best
          record_all_orig=np.append(record_all_orig,orig_seq[improve],axis=0) # record original
          record_seed_act=np.append(record_seed_act,activation_init[improve],axis=0) #
          record_all_seq=np.append(record_all_seq,oh[improve,:,:,:],axis=0) # extends best inptus
          record_all_act=np.append(record_all_act,oh_activation[improve],axis=0) 
          record_trim=np.append(record_trim,trim1[improve,:],axis=0) 
          best_iter[improve]=count
        holdcnt[improve]=0
        holdcnt[~improve]=holdcnt[~improve]+1 # how many iterations in best
        mask=(holdcnt>=buff_range) # anything present longer than buffer

        if sum(mask)==tryi or count>1000:
          print('Converge at',count)
          print('Activation',np.mean(activation_all))
          print('Converged',sum(mask))
          print('Loss',np.mean(loss_all))
          print('Best score',np.mean(best_activation))
          out_seq=mat2oh(record_all_seq,record_trim)
          oh_seq=np.append(oh_seq,out_seq,axis=0)
          break

        count=count+1

      best_map=np.append(best_map,best_input,axis=0)
      best_act=np.append(best_act,best_activation,axis=0)
      best_it=np.append(best_it,best_iter,axis=0)
      record_all=np.append(record_all,record_all_seq,axis=0)
      record_act=np.append(record_act,record_all_act,axis=0)
      record_seed=np.append(record_seed,record_all_orig,axis=0)
      record_sact=np.append(record_sact,record_seed_act,axis=0)
    
    tsv=np.concatenate((np.arange(oh_seq.shape[0]).reshape(oh_seq.shape[0],1),oh_seq.reshape(oh_seq.shape[0],1),record_seed.reshape(record_seed.shape[0],1),record_act.reshape(record_act.shape[0],1),record_sact.reshape(record_sact.shape[0],1)),axis=1)
    np.savetxt(f'./generated/{model_name}/{outname}',tsv,fmt='%s',delimiter='\t')
    print("SAVED!")

initial activation -0.50330615
initial loss -0.50330615
Converge at 20
Activation 3.1732643
Converged 2000
Loss 3.1732643
Best score -0.478419983625412
initial activation -0.5061514
initial loss -0.5061514
Converge at 33
Activation 3.169825
Converged 2000
Loss 3.169825
Best score -0.4850732272267342
initial activation -0.510785
initial loss -0.510785
Converge at 27
Activation 3.2020936
Converged 2000
Loss 3.2020936
Best score -0.48795002146065236
initial activation -0.4591653
initial loss -0.4591653
Converge at 23
Activation 3.3631647
Converged 728
Loss 3.3631647
Best score -0.4407800175658949
SAVED!
initial activation -0.50330615
initial loss -0.50330615
Converge at 19
Activation 6.9865685
Converged 2000
Loss 6.9865685
Best score -0.4780614365339279
initial activation -0.5061514
initial loss -0.5061514
Converge at 21
Activation 6.9442334
Converged 2000
Loss 6.9442334
Best score -0.48514808270335197
initial activation -0.510785
initial loss -0.510785
Converge at 20
Activation 7.002925
